##### Домашнее задание к лекции «Случайные события. Случайные величины»

Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб., после чего выпадает случайное целое число, равномерно распределенное от 0 до 999. При некоторых значениях числа игрок получает выигрыш (см. справа)

Выгодна ли игра игроку?

Сколько в среднем приобретает
или теряет игрок за одну игру?

1. 777: 200 руб.
- 999: 100 руб.
- 555: 50 руб.
- 333: 15 руб.
- 111: 10 руб.
- *77: 5 руб.
- **7: 3 руб.
- *00: 2 руб.
- **0: 1 руб.

\* – любая цифра

Дополнительное задание повышенной сложности. Теоретически рассчитать средний выигрыш (проигрыш) и сравнить с результатами моделирования.


In [1]:
import re
import numpy as np
import pandas as pd

##### Решение по основному заданию

In [2]:
# Решение 

class onearmed_bandit:
    """
    Класс имитирует один подход игрока к автомату 
    repeat_count - кол-во попыток за один подход
    coast_one - стоимость 1-й игры
    player_number - счетчик игроков (экземпляров класса)
    generate_default - выпавшие числа, который возможно задать по-умолчанию, инеча формируется случайная последовательность
    counter_of_winning_combinations - статистика игры, выпавшие числа
    MAX_NUM - максимальное число
    WIN1, WIN10, WIN100 - словарь выиграшных комбинаций, остатки от делания на XXX (WINX, WINXX,WINXXX)
    """
    player_number = 0
    MAX_NUM    = 999
    
    WIN1   = {777: 200, 999: 100, 555: 50, 333: 15, 111: 10}
    WIN10  = {7:  3, 0: 1}
    WIN100 = {77: 5, 0: 2}
    
    TEST_DICT = {777:200,999:100,555:50,333:15,111:10,577:5,307:3,300:2,410:1}
    
    def __init__(self, repeat_count_ = 1000, coast_one_ = 1, generate_default_ = []):
        self.repeat_count = repeat_count_
        self.coast_one = coast_one_
        self.player_number += 1
        self.generate_default = generate_default_
        self.counter_of_winning_combinations = {'777':[0, 0],'999':[0, 0],'555':[0, 0], '333':[0, 0], '111':[0, 0],
                                                '*77':[0, 0],'**7':[0, 0],'*00':[0, 0], '**0':[0, 0], '***':[0, 0]}
        self.statistics_arr = np.array([[],[]])
    
    def check_win(self, num_win1, num_win10, num_win100):
        """
        Проверка выиграшной комбинации
        """
        if num_win1 in self.WIN1.keys(): 
            ret = self.WIN1[num_win1]
            self.counter_of_winning_combinations[str(num_win1)][0] += 1
            self.counter_of_winning_combinations[str(num_win1)][1] += ret
            
        elif num_win100 in self.WIN100.keys(): 
            ret = self.WIN100[num_win100]
            if num_win100 == 0:
                str_num_win100 = "*00"
            else:
                str_num_win100 = '*' + str(num_win100)
            self.counter_of_winning_combinations[str_num_win100][0] += 1
            self.counter_of_winning_combinations[str_num_win100][1] += ret
                
        elif num_win10  in self.WIN10.keys() : 
            ret = self.WIN10[num_win10]
            str_num_win10 = '**' + str(num_win10)
            self.counter_of_winning_combinations[str_num_win10][0] += 1
            self.counter_of_winning_combinations[str_num_win10][1] += ret
        else:
            self.counter_of_winning_combinations['***'][0] += 1
            ret = 0

        return ret
    
    def test(self):
        """
        Тестирование основной функции класса "check_win" на основе тетовового словаря TEST_DICT
        результат словарь = {число:[сумма выигрыша факт, сумма выигрыша прогноз, результат проверки]}
        """
        ret = {}
        for key, value in self.TEST_DICT.items():
            summa_win_check = self.check_win(key,key%10,key%100)
            ret[key]  = [value, 
                         summa_win_check,
                        (value == summa_win_check)]
        return ret
    
    def game_statistics(self):
        """
        Статистика игры (statistics_arr). Возращается матрица (repeat_count x 3 ): порядковый номер, случайное число, выирыш в руб.
        Одной попыткой считаем генерация случайного числа от 0 до 999, есть вариат генерации 3-х случайных значений от 0 до 9,
        но он в данном решении не рассматривается.
        """
        #Без otypes первый раз функции осуществляется 2 раза
        v_check_win = np.vectorize(self.check_win, otypes = ['int32'])

        if len(self.generate_default) == 0:
            self.generate_arr = np.random.randint(0, self.MAX_NUM + 1, self.repeat_count)
        else:
            self.generate_arr = np.array(self.generate_default)
        
        rezult_arr = v_check_win(self.generate_arr, self.generate_arr%10, self.generate_arr%100)
        self.statistics_arr = np.array([self.generate_arr, rezult_arr])
        
        return self.statistics_arr
    
    def main(self):
        """
        Реузльтат игры возвращается list [затраченныя сумма (количество попыток x стоимость 1-й игры), сумма выпгрыша]
        """
        self.game_statistics()
        return [self.repeat_count*self.coast_one, 
                self.statistics_arr[1].sum()]

In [3]:
#Тестирование класса
test_onearmed_bandit = onearmed_bandit(10)
test_onearmed_bandit.test()

{777: [200, 200, True],
 999: [100, 100, True],
 555: [50, 50, True],
 333: [15, 15, True],
 111: [10, 10, True],
 577: [5, 5, True],
 307: [3, 3, True],
 300: [2, 2, True],
 410: [1, 1, True]}

In [4]:
#Имитируем игру 3-х игроков в течении 1 года.

play_list = [ {'Barack Hussein Obama':[onearmed_bandit(1000), []], 
               'George Walker Bush':[onearmed_bandit(500),  []], 
               'Donald John Trump':[onearmed_bandit(100),  []]} ]

rezult = {-1: 'ВЫГОДНО', 0: 'ПРИ СВОИХ', 1: 'УБЫТОЧНО'}

DAYS = 365

for player in play_list[0].keys():
    for day in range(DAYS):
        play_list[0][player][1].append(play_list[0][player][0].main()[1])
    
    coast = play_list[0][player][0].repeat_count * play_list[0][player][0].coast_one
    winning = sum(play_list[0][player][1])
    
    mean_result_play = round( (DAYS*coast - winning)/DAYS/play_list[0][player][0].repeat_count, 2)
    
    print(f'Игрок {player} в течение {DAYS} дней тратил на однорукого бандита по {coast} руб.\n\
Общая сумма расходов {DAYS*coast} руб. Выгрыш {winning} руб.\n\n\
Результат игр "{rezult[np.sign(mean_result_play)]}" {abs(mean_result_play)} руб. за 1 попытку в день.\n\
------------------------------------------------------------------------ \n\
Статистика игр:\n {play_list[0][player][0].counter_of_winning_combinations }\n' )

        

Игрок Barack Hussein Obama в течение 365 дней тратил на однорукого бандита по 1000 руб.
Общая сумма расходов 365000 руб. Выгрыш 287591 руб.

Результат игр "УБЫТОЧНО" 0.21 руб. за 1 попытку в день.
------------------------------------------------------------------------ 
Статистика игр:
 {'777': [345, 69000], '999': [367, 36700], '555': [371, 18550], '333': [317, 4755], '111': [358, 3580], '*77': [3310, 16550], '**7': [32864, 98592], '*00': [3613, 7226], '**0': [32638, 32638], '***': [290817, 0]}

Игрок George Walker Bush в течение 365 дней тратил на однорукого бандита по 500 руб.
Общая сумма расходов 182500 руб. Выгрыш 144952 руб.

Результат игр "УБЫТОЧНО" 0.21 руб. за 1 попытку в день.
------------------------------------------------------------------------ 
Статистика игр:
 {'777': [181, 36200], '999': [186, 18600], '555': [166, 8300], '333': [202, 3030], '111': [192, 1920], '*77': [1618, 8090], '**7': [16206, 48618], '*00': [1839, 3678], '**0': [16516, 16516], '***': [145394, 0]}

И

##### Решение по дополнительному заданию

In [5]:
#Определяем все выигрышные комбинации 
decision_onearmed_bandit = onearmed_bandit(1,1,list(range(0,1000)))
decision_onearmed_bandit.main()
decision_onearmed_bandit.counter_of_winning_combinations

{'777': [1, 200],
 '999': [1, 100],
 '555': [1, 50],
 '333': [1, 15],
 '111': [1, 10],
 '*77': [9, 45],
 '**7': [90, 270],
 '*00': [10, 20],
 '**0': [90, 90],
 '***': [796, 0]}

In [6]:
#Расчитываем выгоду

summa_win  = sum([item[1] for key, item in decision_onearmed_bandit.counter_of_winning_combinations.items()])
summa_cost = len(decision_onearmed_bandit.statistics_arr[0]) * decision_onearmed_bandit.coast_one 
rezult_str = rezult[np.sign(summa_cost - summa_win)]

print(f'Если мы потратим на игру {summa_cost} руб. мы выиграем {summa_win} руб., результат игры "{rezult_str}" \
{(summa_cost - summa_win)/summa_cost} руб. на 1 руб. расходов.')


Если мы потратим на игру 1000 руб. мы выиграем 800 руб., результат игры "УБЫТОЧНО" 0.2 руб. на 1 руб. расходов.
